# S.VI.B: FIMS

This notebook plots and analyzes the results shown in Figure 7a -- 7f of Section VI.B: "FIMS."

To simplify reproducability of the artifact, we have created three slightly-modified copies of the FIMS software pipeline presented in:

> D. Wang, J. Zhang, J. Buhler, and J. Wang, “Real-time analysis of aerosol size distributions with the fast integrated mobility spectrometer (FIMS),” in 41st Conference of American Association for Aerosol Research (AAAR), Oct. 2023. [Online]. Available: https://aaarabstracts.com/2023/view_abstract.php?pid=752 

These include:
* `fims_execution_time_profile`: Obtains execution times with `getrusage()` for each job iteration of all tasks.
* `fims_elasticity_values_assignment`: 
* `fims_evaluate_online_adjustment`:

## Build and Prepare System

### Enable `sudo` Commands from Jupyter Notebook

Several of the scripts in this notebook have to be run as `root`, e.g., to set real-time priorities. We use the `getpass` Python module to prompt for a password, then invoke `sudo` using `os.system`.

In [ ]:
import getpass
import os
password = getpass.getpass()
def run_sudo(cmd) :
    return os.system('echo %s | sudo -S %s' % (password, cmd))
if run_sudo("su") != 0 :
    print("Entered password incorrect! Please run this cell again.")


### Install OpenCV

***Note: The following script can be skipped on the provided virtual machine***

Refer to the OpenCV installation tutorial for more detailed instrutions:
https://docs.opencv.org/4.7.0/d7/d9f/tutorial_linux_install.html

In [ ]:
%%sh

# Download and unpack sources
wget -O opencv.zip https://github.com/opencv/opencv/archive/4.7.0.zip
wget -O opencv_contrib.zip https://github.com/opencv/opencv_contrib/archive/4.7.0.zip
unzip opencv.zip
unzip opencv_contrib.zip

# Create build directory and switch into it
mkdir -p build && cd build

# Configure
cmake -DOPENCV_EXTRA_MODULES_PATH=../opencv_contrib-4.7.0/modules ../opencv-4.7.0

# Build
make -j

In [ ]:
# Install
run_sudo("make install")

### Compile FIMS Code

***Note: The following script can be skipped on the provided virtual machine***

In [ ]:
%%sh

# Compile Code for Profiling Execution Times
cd fims_execution_time_profile
mkdir -p build && cd build
cmake ..
make -j

# Compile Code for Assigning Elasticity Values
cd ../../fims_elasticity_values_assignment
mkdir -p build && cd build
cmake ..
make -j

# Compile Code for Evaluation of Online Adjustment
cd ../../fims_evaluate_online_adjustment
mkdir -p build && cd build
cmake ..
make -j

# Compile Code for Interference Task
cd ../../overhead
make

### Disable Throttling

CPU throttling and real-time scheduler throttling were disabled for our experiments. The following settings were used.

*** Note that this is optional, and not required for correct functionality (though is required to faithfully reproduce timing results on a real system). ***

In [ ]:
run_sudo("sh -c 'echo -1 > /proc/sys/kernel/sched_rt_runtime_us'")
run_sudo("cpufreq-set -r -g performance")

## Profiling Execution Times

We first profile the execution times of each FIMS pipeline task. We make calls to `getrusage()` when each job completes, measuring the total CPU time (user and system) consumed by the task since the end of the prior job. This accounts for execution of the task's function plus the overhead of context switching and timer handling. To capture worst-case conditional behavior, we force recalculation of the inversion matrix with each iteration of data inversion.

The following code reproduces the results shown in Figure 7d -- 7f.


### Obtain Execution Times

We have already provided the results data in the files under `fims/fims_execution_time_profile/result_/execution_time_on_raspberrypi`. This can be reproduced by running the following.

***Note that a single run takes around 20 minutes.***
Provided data is from 10 runs each over two different aerosol datasets. To save time for the reviewers, the below code only performs a single run by default, though this can be controlled by changing the `run_times` variable.

In [ ]:
# Import Packages

import os
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def enter_folder(folder) :
    print("Currently in", os.getcwd(), "entering", working_folder)
    if os.getcwd()[-1*len(working_folder):] != working_folder :
        os.chdir(working_folder)

def leave_folder(folder) :
    os.chdir("../")
    for _ in range(folder.count("/")):
        os.chdir("../")
    print("Back in", os.getcwd())

In [ ]:
# Change this to perform additional runs
run_times = 1

# Function to rename the folder and create a new one
def manage_folders(old_folder, new_folder, i):
    os.rename(old_folder, f"{old_folder}_{i}")
    os.makedirs(new_folder, exist_ok=True)

working_folder = "fims_execution_time_profile/result_"
executable = "../build/fims"  
result_folder = "run_time"

In [ ]:
# Enter execution time profiling folder
enter_folder(working_folder)

# Create result directory
os.makedirs(result_folder, exist_ok=True)

for i in range(0, run_times, 1):  # define how many time to run the code
    print(f'start running {i} ...')
    if run_sudo(executable) == 0:
        print(f"Run {i+1}: Success")
        # Rename the old folder and create a new one for the next run
        manage_folders(result_folder, result_folder, i)
    else:
        print(f"Run {i+1}: Failure")

    time.sleep(1)  # Optional: pause for 1 second between runs

leave_folder(working_folder)

Results are written into a collection of folders `fims/fims_execution_time_profile/result_/run_time_x`, with x indexing each run. Each file contains space-separated values corresponding to the execution time of each iteration of the corresponding task, in units of milliseconds. We use the times reported by `get_rusage()`, separately writing the system times (files ending in `_sys`) and user times (files ending in `_user`).

### Plot and Analyze Results

The following code plots Figure 7d--7f, showing execution time distributions.

We provide the ability to either plot the original data or data newly-generated by the above steps.

In [ ]:
def plot_execution_times(data, data2) :
    
    for i in range(len(data)):
        data[i] = data[i] + data2[i]

    components = ['image_processing', 'data_inversion', 'hk_reading']
    titles = ['(d) FIMS: Image', '(f) FIMS: Inversion', '(e) FIMS: HK']

    # Define a palette
    colors = sns.color_palette("deep", 5)

    # Use the fivethirtyeight style
    plt.style.use('fivethirtyeight')

    for i, d in enumerate(data):
        fig, ax = plt.subplots(figsize=(4.6, 2.5))
        ax.hist(d, bins=30, density=True, alpha=0.8, edgecolor='black', linewidth=1.5, color=colors[i])
        ax.set_yscale('log')
        ax.set_xlabel('Execution Time (ms)', )
        ax.set_ylabel('Frequency')
        plt.tight_layout()
        fig.savefig(f'{components[i]}_execution_time.png')
        fig.savefig(f'{components[i]}_execution_time.eps')
        plt.show() 
        maximum = np.max(data[i])
        print(f'{titles[i]} (max {maximum}ms)')

#### Plot Original Results

Plots results obtained for the paper from the files under `fims/fims_execution_time_profile/result_/execution_time_on_raspberrypi`.

In [ ]:
# Enter execution time profiling folder
working_folder = "fims_execution_time_profile/result_/execution_time_on_raspberrypi"
enter_folder(working_folder)

# Process userspace time
file_name = 'sparse'
file_name2 = 'dense'
path = ['/image_process_cost_user.txt', '/inversion_time_user.txt', '/hk_reading_time_user.txt']
data = []
for p in path:
    data1 = []
    for i in range(1, 11, 1):
        file = 'run_time_' + file_name + '_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
        if p == '/image_process_cost_user.txt':
            file = 'run_time_' + file_name2 + '_' + str(i) + p
            cur_data = np.loadtxt(file)
            data1 = np.concatenate((data1, cur_data[1:]))
    data.append(data1)

# Process system time
path_sys = ['/image_process_cost_sys.txt', '/inversion_time_sys.txt', '/hk_reading_time_sys.txt']
data2 = []
for p in path_sys:
    data1 = []
    for i in range(1, 11, 1):
        file = 'run_time_' + file_name + '_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
        if p == '/image_process_cost_sys.txt':
            file = 'run_time_' + file_name2 + '_' + str(i) + p
            cur_data = np.loadtxt(file)
            data1 = np.concatenate((data1, cur_data[1:]))

    data2.append(data1)

# Plot
plot_execution_times(data,data2)

leave_folder(working_folder)

#### Plot Newly-Obtained Results

Plots new results obtained from running the above profiling steps.

In [ ]:
# Enter execution time profiling folder
working_folder = "fims_execution_time_profile/result_"
enter_folder(working_folder)

# Process userspace time
path = ['/image_process_time_user.txt', '/inversion_time_user.txt', '/hk_reading_time_user.txt']
data = []
for p in path:
    data1 = []
    for i in range(0, run_times, 1):
        file = 'run_time_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
    data.append(data1)

# Process system time
path_sys = ['/image_process_time_sys.txt', '/inversion_time_sys.txt', '/hk_reading_time_sys.txt']
data2 = []
for p in path_sys:
    data1 = []
    for i in range(0, run_times, 1):
        file = 'run_time_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
    data2.append(data1)

# Plot
plot_execution_times(data,data2)

leave_folder(working_folder)